In [42]:
#import necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import xarray as xr
import scipy

import regionmask

# import regionmask

from scipy import stats
from scipy.stats import pearsonr, linregress

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd

import time
from datetime import datetime, date

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.ticker import MaxNLocator, FuncFormatter
import matplotlib.dates as mdates

import shapefile
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import os

import sys
import os.path
import subprocess
import math
import csv

from netCDF4 import Dataset

#import s3fs
import copy
import json, copy
from pandas import DataFrame
from osgeo import gdal, ogr, osr

In [43]:
# Check if the file was properly saved at the given location (navigate to the folder) and open it from there again
gsfc_halfdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\Data\gsfc.glb_.200204_202406_rl06v2.0_obp-ice6gd_halfdegree.nc')
jpl_halfdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\JPL\GRCTellus.JPL.200204_202408.GLO.RL06.3M.MSCNv04CRI.nc')
csr_quarterdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\CSR\CSR_GRACE_GRACE-FO_RL0603_Mascons_all-corrections.nc')

In [44]:
gsfc_halfdegree_gms

<xarray.Dataset> Size: 487MB
Dimensions:        (lon: 720, lat: 360, time: 234, bounds: 2)
Coordinates:
  * lon            (lon) float64 6kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
  * lat            (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time           (time) datetime64[ns] 2kB 2002-04-18 ... 2024-06-16
Dimensions without coordinates: bounds
Data variables:
    land_mask      (lat, lon) float64 2MB ...
    lwe_thickness  (time, lat, lon) float64 485MB ...
    lon_bounds     (lon, bounds) float64 12kB ...
    lat_bounds     (lat, bounds) float64 6kB ...
    time_bounds    (time, bounds) datetime64[ns] 4kB ...
Attributes: (12/25)
    title:                      NASA GSFC GRACE and GRACE-FO MASCON RL06 v1.0
    summary:                    Monthly gravity solutions from GRACE and GRAC...
    platform:                   GRACE and GRACE-FO
    creator_name:               Bryant Loomis
    creator_email:              bryant.d.loomis@nasa.gov
    creator_url:                https://earth.gsfc.nasa.gov/geo/data/grace-ma...
    ...                         ...
    GIA_removed:                ICE6G-D; Peltier, W. R., D. F. Argus, and R. ...
    geocenter_correction:       JPL TN-13
    C_20_substitution:          TN-14; Loomis et al., 2020, Geophys. Res. Let...
    C_30_substitution:          TN-14; Loomis et al., 2020, Geophys. Res. Let...
    journal_reference:          Loomis et al. 2019, J. Geod., https://doi.org...
    date_created:               date_stamp

In [45]:
import xarray as xr

# Adjust the path and pattern as needed
ds = xr.open_mfdataset(
    r'D:/MSRSGI/Masters_thesis/Data/GRAViS_COSTG_TWS/TWS/*.nc',
    combine='nested',
    concat_dim='time',
    data_vars='minimal',
    coords='minimal',
    compat='override',
    parallel=True
)

In [47]:
ds.to_netcdf('same_combined_costg_GRACE_2002_2016.nc')

In [48]:
costg_combined = xr.open_dataset('same_combined_costg_GRACE_2002_2016.nc')
costg_combined

<xarray.Dataset> Size: 328MB
Dimensions:           (time: 158, lat: 180, lon: 360)
Coordinates:
  * time              (time) datetime64[ns] 1kB 2002-04-18 ... 2016-12-19T12:...
  * lon               (lon) float32 1kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * lat               (lat) float32 720B 89.5 88.5 87.5 ... -87.5 -88.5 -89.5
Data variables:
    tws               (time, lat, lon) float64 82MB ...
    std_tws           (time, lat, lon) float64 82MB ...
    leakage           (time, lat, lon) float64 82MB ...
    model_atmosphere  (time, lat, lon) float64 82MB ...
Attributes: (12/14)
    CDI:              Climate Data Interface version 1.9.8 (https://mpimet.mp...
    Conventions:      CF-1.6
    source:           Monthly COST-G RL01 GRACE/GRACE-FO Level-2B Products, T...
    institution:      GFZ Section 1.3: Earth System Modelling
    title:            Globally Gridded Continental Water Storage Anomalies
    doi:              10.5880/COST-G.GRAVIS_01_L3_TWS
    ...               ...
    project:          GravIS GRACE/GRACE-FO Level-3 Products
    summary:          GRACE-derived time series of gridded terrestrial water ...
    product_release:  01
    product_version:  0005
    date_created:     2023-01-18
    CDO:              Climate Data Operators version 1.9.8 (https://mpimet.mp...

In [49]:
df_costg = costg_combined.tws.to_dataframe()
df_costg

tws
time                lat   lon       
2002-04-18 00:00:00  89.5 0.5    NaN
                          1.5    NaN
                          2.5    NaN
                          3.5    NaN
                          4.5    NaN
...                              ...
2016-12-19 12:00:00 -89.5 355.5  NaN
                          356.5  NaN
                          357.5  NaN
                          358.5  NaN
                          359.5  NaN

[10238400 rows x 1 columns]

In [50]:
df_costg.fillna(0, inplace=True)

In [51]:
df_costg

tws
time                lat   lon       
2002-04-18 00:00:00  89.5 0.5    0.0
                          1.5    0.0
                          2.5    0.0
                          3.5    0.0
                          4.5    0.0
...                              ...
2016-12-19 12:00:00 -89.5 355.5  0.0
                          356.5  0.0
                          357.5  0.0
                          358.5  0.0
                          359.5  0.0

[10238400 rows x 1 columns]

In [52]:
# Assuming df_costg is your DataFrame
df_costg_back2nc = xr.Dataset.from_dataframe(df_costg)

In [54]:
df_costg_back2nc.to_netcdf("same_costg_tws.nc")

In [55]:
costg_back2nc_tws= xr.open_dataset('same_costg_tws.nc')
costg_back2nc_tws

<xarray.Dataset> Size: 82MB
Dimensions:  (time: 158, lat: 180, lon: 360)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-18 ... 2016-12-19T12:00:00
  * lat      (lat) float32 720B 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    tws      (time, lat, lon) float64 82MB ...

In [56]:
costg_back2nc_tws

<xarray.Dataset> Size: 82MB
Dimensions:  (time: 158, lat: 180, lon: 360)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-18 ... 2016-12-19T12:00:00
  * lat      (lat) float32 720B 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
  * lon      (lon) float32 1kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    tws      (time, lat, lon) float64 82MB ...

In [57]:
new_lat = np.arange(89.5, -90, -0.5)
new_lon = np.arange(0.5, 360, 0.5)

ds_halfdeg = costg_back2nc_tws.interp(lat=new_lat, lon=new_lon)
ds_halfdeg.to_netcdf("same_costg_tws_half_degree1.nc")

In [59]:
costg_halfdeg = xr.open_dataset('same_costg_tws_half_degree1.nc')
costg_halfdeg

<xarray.Dataset> Size: 326MB
Dimensions:  (time: 158, lat: 359, lon: 719)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-18 ... 2016-12-19T12:00:00
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.0 -88.5 -89.0 -89.5
  * lon      (lon) float64 6kB 0.5 1.0 1.5 2.0 2.5 ... 358.0 358.5 359.0 359.5
Data variables:
    tws      (time, lat, lon) float64 326MB ...

In [60]:
# Assume `ds` is your xarray dataset
ds_costg = costg_halfdeg  # Replace with your actual dataset

# Shift latitude and longitude
shifted_lon = (ds_costg['lon'] - 0.5) % 360  # Wrap around [0, 360)
shifted_lat = ds_costg['lat'] - 0.5

# Update the dataset with the corrected coordinates
corrected_ds_costg = ds_costg.copy()
corrected_ds_costg = corrected_ds_costg.assign_coords(lon=shifted_lon, lat=shifted_lat)

# Sort longitude to ensure proper ordering after the shift
corrected_ds_costg = corrected_ds_costg.sortby('lon')

# Display the corrected dataset
corrected_ds_costg

<xarray.Dataset> Size: 326MB
Dimensions:  (time: 158, lat: 359, lon: 719)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-18 ... 2016-12-19T12:00:00
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
Data variables:
    tws      (time, lat, lon) float64 326MB ...

In [61]:
# Select the time range from April 2002 to April 2022
subset_costg_halfdegree = corrected_ds_costg.sel(time=slice("2002-04", "2016-03"))
subset_costg_halfdegree

<xarray.Dataset> Size: 314MB
Dimensions:  (time: 152, lat: 359, lon: 719)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-18 ... 2016-03-16T12:00:00
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
Data variables:
    tws      (time, lat, lon) float64 314MB ...

In [62]:
costg_ds = subset_costg_halfdegree
costg_ds = costg_ds.resample(time='M').mean().interpolate_na(dim='time')
costg_ds

C:\Users\sudur\miniconda3\envs\Thesis\Lib\site-packages\xarray\groupers.py:498: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 347MB
Dimensions:  (lon: 719, lat: 359, time: 168)
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Data variables:
    tws      (time, lat, lon) float64 347MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [63]:
costg_lwe_thickness = costg_ds.tws

costg_lwe_thickness_meter = costg_lwe_thickness / 100
costg_lwe_thickness_meter.attrs['units'] = 'meters'  # Optional but good practice
costg_lwe_thickness_meter

<xarray.DataArray 'tws' (time: 168, lat: 359, lon: 719)> Size: 347MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 359, 719))
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Attributes:
    units:    meters

In [64]:
costg_lwe_thickness_meter = costg_lwe_thickness_meter.rename('lwe_thickness')

costg_lwe_thickness_meter 

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 359, lon: 719)> Size: 347MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 359, 719))
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Attributes:
    units:    meters

In [65]:
costg_lwe_thickness_meter 

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 359, lon: 719)> Size: 347MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 359, 719))
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 357.5 358.0 358.5 359.0
  * lat      (lat) float64 3kB 89.0 88.5 88.0 87.5 ... -88.5 -89.0 -89.5 -90.0
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Attributes:
    units:    meters

In [67]:
import numpy as np

# Existing coordinates
old_lat = costg_lwe_thickness_meter['lat'].values
old_lon = costg_lwe_thickness_meter['lon'].values

new_lat = np.insert(old_lat, 0, 89.5) #append 89.5 at the left of 89
new_lon = np.append(old_lon, 359.5)  # Append 359.5 at the end

In [ ]:

new_shape = (costg_lwe_thickness_meter.sizes['time'], len(new_lat), len(new_lon))
new_data = np.zeros(new_shape, dtype=costg_lwe_thickness_meter.dtype)

# Create new DataArray
new_da = xr.DataArray(
    new_data,
    coords={'time': costg_ds['time'], 'lat': new_lat, 'lon': new_lon},
    dims=['time', 'lat', 'lon'],
    # name=costg_ds.name,
    attrs=costg_ds.attrs
)

In [ ]:
new_da

In [68]:
# Assign old data into the corresponding slice of the new array
new_da.loc[dict(
    time=costg_lwe_thickness_meter['time'],
    lat=old_lat,
    lon=old_lon
)] = costg_lwe_thickness_meter.values
new_da

<xarray.DataArray (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 360, 720))
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [72]:
# After assigning values to new_da
new_da = new_da.rename("lwe_thickness")
new_da

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 360, 720))
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [74]:
new_da.to_netcdf(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New1_costg3_pdsmc_tws1_meter.nc')

In [76]:
costg=xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New1_costg3_pdsmc_tws1_meter.nc')
costg

<xarray.Dataset> Size: 348MB
Dimensions:        (time: 168, lat: 360, lon: 720)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
  * lat            (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -89.0 -89.5 -90.0
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB ...

In [84]:
costg.attrs['units'] = 'meters'
costg

<xarray.Dataset> Size: 348MB
Dimensions:        (time: 168, lat: 360, lon: 720)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
  * lat            (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -89.0 -89.5 -90.0
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    units:    meters

In [77]:
costg.lwe_thickness.values

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [78]:
# gdf = gpd.read_file(r'D:/MSRSGI/Masters_thesis/Data/HMA_Region_Shapefiles/Regions_shapefiles/HMA_region_annoted_final.shp')
# gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS
# gdf_4326 

# 1. Load and prepare the shapefile
gdf = gpd.read_file(r'D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_TotalHMA_boundary1_22.shp')
gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS
gdf_4326 

,Primary_ID,Primary__1,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,Eastern Hindu Kush (8),111147.061871,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,Western Himalaya (10),211514.200081,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,Eastern Himalaya (12),166943.561268,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,Central Himalaya (11),270268.094462,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,Karakoram (9),92830.485352,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,Western Pamir (6),124880.500694,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,Pamir Alay (5),83444.425531,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,Northern/Western Tien Shan (4),205724.916795,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,Dzhungarsky Alatau (1),39543.166059,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,Western Kunlun Shan (13),132813.049169,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [79]:
dropped_gdf_4326 = gdf_4326.drop(['Primary__1'], axis=1)
dropped_gdf_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,111147.061871,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,211514.200081,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,166943.561268,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,270268.094462,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,92830.485352,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,124880.500694,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,83444.425531,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,205724.916795,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,39543.166059,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,132813.049169,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [80]:
gdf1 = gpd.read_file(r"D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_SingleHMAboundarythatincludes1to22.shp")
gdf1_4326 = gdf1.to_crs('EPSG:4326')  # Reproject to match dataset CRS

dropped_gdf1_4326 = gdf1_4326.drop(['Primary__1'], axis=1)  #already done , so comes weror wehn done
dropped_gdf1_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,High Mountain Asia Region,3.993978e+06,3.993978e+12,23,"POLYGON ((92.26394 39.3763, 92.24847 39.32136,..."


In [81]:
gdf_concat = pd.concat([dropped_gdf_4326, dropped_gdf1_4326 ], axis=0, ignore_index=True)
gdf_concat

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,1.111471e+05,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,2.115142e+05,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,1.669436e+05,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,2.702681e+05,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,9.283049e+04,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,1.248805e+05,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,8.344443e+04,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,2.057249e+05,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,3.954317e+04,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,1.328130e+05,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [87]:
# ds = xr.open_dataset(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New1_costg1_pdsmc_tws1_meter.nc')
# da = ds['lwe_thickness']  # Get the DataArray
# da

ds = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New1_costg3_pdsmc_tws1_meter.nc')
da = ds['lwe_thickness']  # Get the DataArray
da

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
[43545600 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [90]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
from shapely.geometry import mapping

# 1. Load and prepare the shapefile
gdf = gpd.read_file(r'D:/MSRSGI/Masters_thesis/Data/HMA_Region_Shapefiles/Regions_shapefiles/HMA_region_annoted_final.shp')
gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS


# 2. Load the lwe_thickness dataset 
# ds = xr.open_dataset(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/costg3_pdsmc_tws1_meter.nc')
# da = ds['lwe_thickness']  # Get the DataArray
ds = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New1_costg3_pdsmc_tws1_meter.nc')
da = ds['lwe_thickness']  # Get the DataArray

# 3. Define the provided functions
def area_weighting(grid_resolution):
    """
    Calculate the area of each grid, globally, corresponding to the latitudes and longitudes.
    """
    longitude_grid = np.linspace(0, 359+(1-grid_resolution), int(360/grid_resolution), dtype='float')
    latitude_grid = np.linspace(0, 179+(1-grid_resolution), int(180/grid_resolution), dtype='float')
    longitude_grid1 = np.linspace(1*grid_resolution, 360, int(360/grid_resolution), dtype='float')
    latitude_grid1 = np.linspace(1*grid_resolution, 180, int(180/grid_resolution), dtype='float')
    
    lambd,theta = np.meshgrid(longitude_grid, latitude_grid)  
    lambd1,theta1 = np.meshgrid(longitude_grid1, latitude_grid1)
    
    delta_latitude = np.sin(np.deg2rad(90-theta)) - np.sin(np.deg2rad(90-theta1))
    delta_longitude = (lambd1 - lambd) * np.pi/180

    # Area of each grid in m²
    area = (6378.137**2) * pow(10,6) * (np.multiply(delta_latitude, delta_longitude))
    return area

def Basinaverage(temp, gs, shp_basin, basin_area):
    """
    Calculate the basin average of the total water storage (TWS).
    """
    # Apply area weighting to the data
    temp_weighted = temp.copy()
    temp_weighted = temp * area_weighting(gs)  # Convert to volume
    
    # Set spatial dimensions and CRS
    basin_tws = temp_weighted.rio.write_crs("EPSG:4326", inplace=True)
    basin_tws = basin_tws.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
            
    # Clip data to basin shape
    basin_tws = basin_tws.rio.clip(shp_basin.geometry.apply(mapping), shp_basin.crs, drop=True)
    
    # Calculate basin average TWS
    basin_avg_tws = basin_tws.sum(dim=('lon','lat'), skipna=True) / basin_area
    
    # Clean up spatial references
    if 'spatial_ref' in basin_avg_tws.coords:
        basin_avg_tws = basin_avg_tws.drop_vars('spatial_ref')
    
    return basin_avg_tws

# 4. Process each region
results = []
grid_resolution = 0.5  # Grid resolution in degrees

#for idx, region in gdf_4326.iterrows():
for idx, region in gdf_concat.iterrows():
    region_name = region.Primary_ID
    region_area = region.Shape_Area  # Exact region area from shapefile
    number_region = region.Region_No #latest
    
    #  GeoDataFrame
    region_gdf = gpd.GeoDataFrame({
        'geometry': [region.geometry],
        'Primary_ID': [region_name],
        'Region_No': [number_region],
        'Shape_Area': [region_area]
    }, crs=gdf_concat.crs)   #gdf_4326 is replaced by gdf_concat
    
    # Calculate the basin average using the reference function
    basin_avg = Basinaverage(da, grid_resolution, region_gdf, region_area)
    
    # # Add region metadata
    # basin_avg = basin_avg.assign_attrs({
    #     'region_name': region_name,
    #     #'region_area_m2': float(region_area),
    #     'region_area_m2': region_area,
    #     'region_id': number_region
    # })

    meta = xr.Dataset({
    'region_id': result_ds.region_id,
    'region_area_m2': result_ds.region_area_m2}, coords={'region': result_ds.region})

    # print(meta.to_dataframe())
    
    basin_avg = basin_avg.expand_dims(region=[region_name])
    basin_avg = basin_avg.assign_coords({
    'region_area_m2': ('region', [region_area]),
    'region_id': ('region', [number_region])})
    results.append(basin_avg)

# 5. Combine all regions into a single dataset
result_ds = xr.concat(results, dim='region')

In [91]:
result_ds

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[ 0.06933469,  0.10023189,  0.05520419, ..., -0.04243037,
        -0.0055811 ,  0.04357951],
       [ 0.1361211 ,  0.13564183,  0.10689195, ..., -0.05810992,
        -0.04023699, -0.01396686],
       [ 0.06954845,  0.11764553,  0.16321819, ..., -0.24478753,
        -0.2418514 , -0.23733805],
       ...,
       [ 0.07951855,  0.1014095 ,  0.09923423, ..., -0.08823382,
        -0.08093793, -0.0961507 ],
       [-0.01109928,  0.02024423,  0.00947053, ..., -0.0299557 ,
        -0.03517954, -0.04345673],
       [ 0.01105695,  0.01701944,  0.02171163, ..., -0.05382648,
        -0.05199422, -0.055578  ]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [94]:
result_ds_mm = result_ds * 1000
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  69.33468627,  100.23188625,   55.20419367, ...,  -42.43037365,
          -5.58110384,   43.57951126],
       [ 136.12109649,  135.64183348,  106.89195416, ...,  -58.10992079,
         -40.23699213,  -13.96686304],
       [  69.54845013,  117.64552783,  163.21818657, ..., -244.78752682,
        -241.85139546, -237.33805374],
       ...,
       [  79.5185544 ,  101.40949584,   99.23423305, ...,  -88.23382256,
         -80.93793024,  -96.15069892],
       [ -11.09927662,   20.24423423,    9.47053091, ...,  -29.95570049,
         -35.17953545,  -43.45672557],
       [  11.05695435,   17.01943538,   21.7116273 , ...,  -53.82647885,
         -51.99421891,  -55.57800054]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [95]:
result_ds_mm.attrs['units'] = 'mm'
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  69.33468627,  100.23188625,   55.20419367, ...,  -42.43037365,
          -5.58110384,   43.57951126],
       [ 136.12109649,  135.64183348,  106.89195416, ...,  -58.10992079,
         -40.23699213,  -13.96686304],
       [  69.54845013,  117.64552783,  163.21818657, ..., -244.78752682,
        -241.85139546, -237.33805374],
       ...,
       [  79.5185544 ,  101.40949584,   99.23423305, ...,  -88.23382256,
         -80.93793024,  -96.15069892],
       [ -11.09927662,   20.24423423,    9.47053091, ...,  -29.95570049,
         -35.17953545,  -43.45672557],
       [  11.05695435,   17.01943538,   21.7116273 , ...,  -53.82647885,
         -51.99421891,  -55.57800054]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23
Attributes:
    units:    mm

In [96]:
meta.to_dataframe()

,region_area_m2,region_id
region,,
Eastern Hindu Kush,1.111471e+11,8
Western Himalaya,2.115142e+11,10
Eastern Himalaya,1.669436e+11,12
Central Himalaya,2.702681e+11,11
Karakoram,9.283049e+10,9
Western Pamir,1.248805e+11,6
Pamir Alay,8.344443e+10,5
Northern/Western Tien Shan,2.057249e+11,4
Dzhungarsky Alatau,3.954317e+10,1


In [97]:
# 6. Save to NetCDF
result_ds_mm.to_netcdf(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averages_costg_mm_End.nc')

In [98]:
# Load the dataset
ds_costg = xr.open_dataset(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averages_costg_mm_End.nc')

# Rearrange dimensions to (time, region)
da_costg = ds_costg['lwe_thickness'].transpose('time', 'region')

# Convert to pandas DataFrame
df_costg = da_costg.to_pandas()  # time as index, region names as columns
df_costg

region,Eastern Hindu Kush,Western Himalaya,Eastern Himalaya,Central Himalaya,Karakoram,Western Pamir,Pamir Alay,Northern/Western Tien Shan,Dzhungarsky Alatau,Western Kunlun Shan,...,Tibetan Interior Mountains,Tanggula Shan,Eastern Tibetan Mountains,Qilian Shan,Eastern Kunlun Shan,Altun Shan,Eastern Tien Shan,Central Tien Shan,Eastern Pamir,High Mountain Asia Region
time,,,,,,,,,,,,,,,,,,,,,
2002-04-30,69.334686,136.121096,69.548450,97.997670,16.783038,58.615274,61.395764,43.381782,46.102289,-52.162673,...,-76.767095,-37.347057,-52.904445,-12.310697,-82.253688,-19.882599,88.388404,79.518554,-11.099277,11.056954
2002-05-31,100.231886,135.641833,117.645528,94.848581,41.136770,103.068410,89.567156,63.485458,57.221536,-41.678457,...,-83.570476,-30.776838,-69.488500,-24.660005,-78.377365,-19.315917,103.908524,101.409496,20.244234,17.019435
2002-06-30,55.204194,106.891954,163.218187,122.254257,14.197910,64.310955,57.866299,50.879507,47.092907,-41.959246,...,-76.567398,-17.037640,-56.558090,-12.710320,-73.639605,-19.011062,96.359039,99.234233,9.470531,21.711627
2002-07-31,8.675578,77.183746,210.309934,150.573455,-13.638913,24.261586,25.108747,37.853358,36.626657,-42.249395,...,-69.330884,-2.840468,-43.196666,-0.362313,-68.743920,-18.696046,88.557903,96.986461,-1.662296,26.560226
2002-08-31,-37.853038,47.475537,257.401681,178.892654,-41.475736,-15.787784,-7.648806,24.827209,26.160407,-42.539544,...,-62.094369,11.356703,-29.835243,11.985695,-63.848235,-18.381030,80.756768,94.738690,-12.795123,31.408824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,-57.173639,-79.100935,-156.855706,-112.422568,-45.897295,-89.440205,-72.631333,-81.844875,-45.859479,27.336318,...,-10.777203,-71.536067,-6.289980,-14.413137,21.509030,13.176479,-69.983156,-100.181148,-48.750368,-48.753525
2015-12-31,-34.157123,-84.267482,-234.953125,-158.153946,-34.288330,-61.390916,-49.712871,-60.261627,-18.573853,31.680613,...,-21.612701,-96.977980,-25.354308,-21.700179,10.406590,8.230543,-62.182828,-89.211341,-42.086874,-62.136437
2016-01-31,-42.430374,-58.109921,-244.787527,-170.352499,-11.276091,-71.629659,-55.750508,-46.288446,-6.705156,44.240959,...,-29.036881,-62.640850,3.167076,4.345751,21.792350,24.299419,-55.027370,-88.233823,-29.955700,-53.826479


In [99]:
# Your region metadata
region_metadata_s = pd.DataFrame({
    'region': [
        'Eastern Hindu Kush', 'Western Himalaya', 'Eastern Himalaya', 'Central Himalaya', 'Karakoram',
        'Western Pamir', 'Pamir Alay', 'Northern/Western Tien Shan', 'Dzhungarsky Alatau', 'Western Kunlun Shan', 'Nyainqentanglha', 'Gangdise Mountains', 'Hengduan Shan',
        'Tibetan Interior Mountains', 'Tanggula Shan', 'Eastern Tibetan Mountains', 'Qilian Shan', 'Eastern Kunlun Shan', 'Altun Shan',
        'Eastern Tien Shan', 'Central Tien Shan', 'Eastern Pamir', 'High Mountain Asia Region'
    ],
    'region_id': [8, 10, 12, 11, 9, 6, 5, 4, 1, 13, 18, 17, 20, 16, 19, 21, 22, 14, 15, 2, 3, 7, 23]
})

# Sort by region_id
region_metadata_sorted_s = region_metadata_s.sort_values(by='region_id')
sorted_columns_s = region_metadata_sorted_s['region'].tolist()

df_s = df_costg[sorted_columns_s]

df_s.index.name = 'time'
df_s.to_csv(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_COSTG.csv', index=True)